# PyTorch: Rango dinámico, precisión y tiempos (FP16, FP32, FP64, bfloat16)

## Introducción

En esta notebook exploraremos los diferentes tipos de datos numéricos que PyTorch utiliza para representar tensores. Comprenderemos las diferencias fundamentales entre precisión y rango dinámico, y cómo estas diferencias afectan el rendimiento computacional.

**Objetivos:**
- Entender la diferencia entre rango dinámico y precisión
- Comparar los tipos de datos más comunes: FP16, FP32, FP64 y bfloat16
- Medir el impacto en memoria y velocidad de cada tipo
- Identificar cuándo usar cada tipo de dato en la práctica

## Fundamentos teóricos: Representación de números en coma flotante

Un número en coma flotante se representa matemáticamente como:

$$x = (-1)^{signo} \times 1.mantisa \times 2^{exponente}$$

Donde cada componente cumple una función específica:

- **Exponente → rango dinámico**: Determina qué tan grandes o pequeños pueden ser los números sin causar overflow/underflow
- **Mantisa → precisión**: Controla cuántos dígitos significativos se conservan en la representación

### Distribución de bits por tipo

Los diferentes tipos de datos asignan bits de manera distinta:

- **FP16**: 1 signo + 5 exponente + 10 mantisa → rango limitado, precisión moderada
- **FP32**: 1 signo + 8 exponente + 23 mantisa → buen equilibrio entre rango y precisión
- **FP64**: 1 signo + 11 exponente + 52 mantisa → máximo rango y precisión
- **bfloat16**: 1 signo + 8 exponente + 7 mantisa → mismo rango que FP32, menor precisión


In [5]:
import torch

In [6]:
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.backends.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

In [7]:
dtype_dic = {
    "float16": torch.float16,
    "float32": torch.float32,
    "float64": torch.float64,
    "bfloat16": torch.bfloat16
}

Cada tipo de dato tiene limitaciones específicas que determinan su aplicabilidad:

In [9]:
def show_finfo(dtype, name):
    try:
        info = torch.finfo(dtype)
        print(f"{name:<10} | bits={info.bits} | eps={info.eps:.2e} | min={info.min:.2e} | max={info.max:.2e}")
    except Exception as e:
        print(f"{name:<10} | no disponible -> {e}")

for name, dtype in dtype_dic.items():
    show_finfo(dtype, name)

float16    | bits=16 | eps=9.77e-04 | min=-6.55e+04 | max=6.55e+04
float32    | bits=32 | eps=1.19e-07 | min=-3.40e+38 | max=3.40e+38
float64    | bits=64 | eps=2.22e-16 | min=-1.80e+308 | max=1.80e+308
bfloat16   | bits=16 | eps=7.81e-03 | min=-3.39e+38 | max=3.39e+38


**Interpretación de resultados:**

- **bits**: Cantidad total de bits utilizados para la representación.
- **eps**: Representa la precisión máxima del tipo. Es el menor número positivo tal que $1.0 + \text{eps} \neq 1.0$
- **min/max**: Indican el rango dinámico, es decir, los valores extremos que se pueden representar sin overflow

### Demostración de pérdida de precisión

El problema clásico de la aritmética de punto flotante:

In [11]:
# Problema clásico: 0.1 + 0.2 ≠ 0.3
for name, dtype in dtype_dic.items():
    a = torch.tensor(0.1, dtype=dtype)
    b = torch.tensor(0.2, dtype=dtype)
    resultado = a + b
    error = abs(resultado.item() - 0.3)
    print(f"{name}: 0.1 + 0.2 = {resultado:.10f} | error = {error:.2e}")

float16: 0.1 + 0.2 = 0.2998046875 | error = 1.95e-04
float32: 0.1 + 0.2 = 0.3000000119 | error = 1.19e-08
float64: 0.1 + 0.2 = 0.3000000000 | error = 5.55e-17
bfloat16: 0.1 + 0.2 = 0.3007812500 | error = 7.81e-04


Este experimento ilustra cómo la precisión limitada de cada tipo afecta operaciones aparentemente simples. La diferencia con el resultado esperado (0.3) varía según el tipo de dato utilizado.

### Overflow: Desbordamiento numérico

Cuando un cálculo produce un resultado que excede el rango máximo del tipo de dato:

In [6]:
# Overflow: cuando el número es demasiado grande
for name, dtype in dtype_dic.items():
    info = torch.finfo(dtype)
    x = torch.tensor(info.max, dtype=dtype)
    print(f"\n{name}:")
    print(f"  max = {x}")
    print(f"  max + 10 = {x + 10}")
    print(f"  max * 2 = {x * 2}")  # ¡Overflow!
   


float16:
  max = 65504.0
  max + 10 = 65504.0
  max * 2 = inf

float32:
  max = 3.4028234663852886e+38
  max + 10 = 3.4028234663852886e+38
  max * 2 = inf

float64:
  max = 1.7976931348623157e+308
  max + 10 = 1.7976931348623157e+308
  max * 2 = inf

bfloat16:
  max = 3.3895313892515355e+38
  max + 10 = 3.3895313892515355e+38
  max * 2 = inf


> También existe el underflow: cuando un cálculo produce un resultado que está por debajo del rango mínimo del tipo de dato.

## Análisis de rendimiento computacional

### Configuración del experimento

Para evaluar el impacto real de cada tipo de dato, realizaremos multiplicaciones de matrices de gran escala. Este tipo de operación es fundamental en deep learning y permite observar diferencias significativas de rendimiento.

A tener en cuenta: 
- **Warm-up**: Las primeras ejecuciones incluyen tiempo de inicialización de kernels GPU y asignación de memoria. El warm-up elimina estos efectos transitorios.
- **Synchronization**: Las GPU ejecutan operaciones de forma asíncrona. `torch.cuda.synchronize()` garantiza que medimos el tiempo real de cómputo, no solo el tiempo de envío de la operación.
- **Múltiples ejecuciones**: El comando `%timeit` ejecuta la operación múltiples veces y calcula estadísticas, proporcionando mediciones más robustas.

In [7]:
size = 2048

print(f"Usando: {DEVICE}\n")
print(f"Tamaño de matrices: {size}x{size}\n")

results = {}

for name, dtype in dtype_dic.items():
    try:
        print(f"{name} ({dtype}):")
        
        # Crear matrices
        A = torch.randn(size, size, dtype=dtype, device=DEVICE)
        B = torch.randn(size, size, dtype=dtype, device=DEVICE)
        
        # Mostrar uso de memoria
        memory_mb = (A.element_size() + B.element_size()) * size * size / (1024**2)
        print(f"  Memoria: {memory_mb:.1f} MB")
        
        # Warm-up: necesario para estabilizar medidas
        for _ in range(10):
            _ = torch.mm(A, B)
        torch.cuda.synchronize() # Esperar a que GPU termine
        
        print(f"{name}:")
        %timeit -n 10 -r 5 torch.mm(A, B); torch.cuda.synchronize()
        print()
        
    except Exception as e:
        print(f"  {name:<10} | no disponible -> {e}")
        print()


Usando: cuda

Tamaño de matrices: 2048x2048

float16 (torch.float16):
  Memoria: 16.0 MB
float16:
381 μs ± 32.1 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)

float32 (torch.float32):
  Memoria: 32.0 MB
float32:
995 μs ± 29.6 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)

float64 (torch.float64):
  Memoria: 64.0 MB
float64:
41.8 ms ± 173 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)

bfloat16 (torch.bfloat16):
  Memoria: 16.0 MB
bfloat16:
354 μs ± 23.4 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)



## Consideraciones para la práctica

### Cuándo usar cada tipo

**FP32**: Punto de partida recomendado
- Desarrollo y prototipado
- Cuando la precisión es crítica
- Modelos que caben en memoria disponible

**FP16/bfloat16**: Optimización de producción
- Modelos grandes que requieren eficiencia
- Hardware con soporte para Tensor Cores
- Entrenamiento distribuido

**FP64**: Casos especializados
- Computación científica de alta precisión
- Simulaciones numéricas críticas
- Validación de algoritmos

### Limitaciones y precauciones

- **Underflow numérico**: FP16 es más susceptible a gradientes muy pequeños
- **Compatibilidad de hardware**: No todos los dispositivos soportan todos los tipos eficientemente
- **Estabilidad numérica**: Algoritmos pueden requerir ajustes para tipos de menor precisión